In [1]:
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

from datetime import date,datetime,timedelta
hoje =  pd.to_datetime(date.today() + timedelta(days=25)) 
final_do_mes = pd.to_datetime(datetime(hoje.year,hoje.month+1,1) - timedelta(seconds=1))

Current working directory was changed to: C:\Users\Vinicius\Documents\GitHub\Previsor\App


In [2]:
files = []
for file in glob.glob("./Queries/v*.txt",):
    a,b = os.path.split(file)
    files.append(b.split(".")[0])
directory = download_db(lista_download=files,force_download=False)


******************** Baixando arquivos *********************

Carregando arquivo de log...
vmonitoramentoleilao já foi baixado no dia: Dia: 30/01/23 - Horário: 12:42:47. Portanto não foi baixado novamente.
vmonitoramentoug já foi baixado no dia: Dia: 30/01/23 - Horário: 12:45:54. Portanto não foi baixado novamente.
vmonitoramentousina já foi baixado no dia: Dia: 30/01/23 - Horário: 12:46:00. Portanto não foi baixado novamente.
vrapeelacesso já foi baixado no dia: Dia: 30/01/23 - Horário: 12:46:09. Portanto não foi baixado novamente.
vrapeelcontratorecurso já foi baixado no dia: Dia: 30/01/23 - Horário: 12:46:32. Portanto não foi baixado novamente.
vrapeelcronograma já foi baixado no dia: Dia: 30/01/23 - Horário: 12:47:39. Portanto não foi baixado novamente.
vrapeelempreendimento já foi baixado no dia: Dia: 30/01/23 - Horário: 12:47:49. Portanto não foi baixado novamente.
vrapeellicenciamento já foi baixado no dia: Dia: 30/01/23 - Horário: 12:48:21. Portanto não foi baixado novamente.


In [3]:
# Lista de colunas que serão usadas várias vezes para a realização de merges entre os dataframes
list_id_data = ['DthEnvio','IdeUsinaOutorga']
list_id_ug = ['IdeUsinaOutorga','NumOperacaoUg']

# Informações de Usinas

## Carrega banco de dados

In [4]:
cols = ["IdeUsinaOutorga",'NomUsina','CodCegFormatado','DatMonitoramento',"DatCanteiroObraRealizado","DatDesvioRioRealizado","DatEnchimentoRealizado","DatConclusaoSisTransRealizado","DatPrevisaoIniciobra","IdcObraParalisada","IdcUsinaMonitorada","DatConcretagemRealizado","DatConclusaoTorresRealizado","DatInicioObraOutorgado","DatSisTransmissaoRealizado","IdcSituacaoObra","IdcSemPrevisao","DatComissionamentoUGRealizado","SigTipoGeracao","DscComercializacaoEnergia","DatInicioObraRealizado","DatMontagemRealizado",'DscJustificativaPrevisao']
df_usina = pd.read_parquet(os.path.join(directory,"vmonitoramentousina.gzip"))[cols]
df_usina.IdcUsinaMonitorada = df_usina.IdcUsinaMonitorada.map({"Sim":"True","Não":False}).astype(bool)
df_usina = df_usina[df_usina.IdcUsinaMonitorada].reset_index(drop=True)

In [5]:
monitoramentoleilao  = pd.read_parquet(os.path.join(directory,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)
monitoramentoleilao = monitoramentoleilao.loc[monitoramentoleilao.groupby('IdeUsinaOutorga').DatInicioSuprimento.idxmin()]
df_usina = pd.merge(df_usina,monitoramentoleilao,how="left",on="IdeUsinaOutorga")

In [6]:
# Carrega BD vrapeelcronograma => rapeel_cronograma
cols = ['IdeUsinaOutorga','CodCeg','DthEnvio','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV']
rename_cols = {
        'DatRealizacaoII' : 'canteiroReal',
        'DatPrevistaAprovacaoIII' : 'prev_IO',
        'DatRealizacaoIII' : 'IO_real',
        'DatRealizacaoIX' : 'DesvRio_real',
        'DatRealizacaoXI' : 'Ench_Real',
        'DatRealizacaoXIII' : 'IOTrans_Real',
        'DatRealizacaoXIV' : 'Conc_Trans_REAL',
        'DatRealizacaoXII' : 'Comiss_Real',
        'DatRealizacaoVII' : 'ME_Real_conc_eol',
        'DatRealizacaoVIII' : 'Conc_Combust_Real'}
        
rapeel_cronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].rename(columns=rename_cols)
rapeel_cronograma = rapeel_cronograma.loc[rapeel_cronograma.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
df_usina = pd.merge(df_usina,rapeel_cronograma,how="left",on="IdeUsinaOutorga")

In [7]:
df_usina["Usina_Sem_Rapeel"] = True
df_usina.loc[df_usina.IdeUsinaOutorga.isin(rapeel_cronograma.IdeUsinaOutorga),'Usina_Sem_Rapeel'] = False

In [8]:
cols = ['DthEnvio','IdeUsinaOutorga','DatConclusaoIII',]
rename_cols = {'DatConclusaoIII':'DatConclusaoACL'}
mercado = pd.read_parquet(os.path.join(directory,"vrapeelcontratorecurso.gzip"))[cols].rename(columns=rename_cols)
mercado = mercado.loc[mercado.groupby("IdeUsinaOutorga").DthEnvio.idxmax()]
del mercado['DthEnvio']
df_usina = pd.merge(df_usina,mercado,on="IdeUsinaOutorga",how="left")

In [9]:
cols = ["DatValidadeIV","DatValidadeI","DthEnvio","IdeUsinaOutorga","DatValidadeV"]
rename_cols = {'DatValidadeIV': 'DatValidadeLI','DatValidadeI': 'DatValidadeLP','DatValidadeV': 'DatValidadeLO'}
ambiental_datas = pd.read_parquet(os.path.join(directory,"vrapeellicenciamento.gzip"))[cols].rename(columns=rename_cols)
ambiental_datas = ambiental_datas.loc[ambiental_datas.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
del ambiental_datas['DthEnvio']
df_usina = pd.merge(df_usina,ambiental_datas,on="IdeUsinaOutorga",how="left")

In [10]:
cols = ["DthEnvio","NomSitContratoI","NomSitContratoIV","NomSituacaoContratoIII","NomSitContratoII","IdeUsinaOutorga"]
rename_cols = {'NomSitContratoI': 'NomSitContratoCCD','NomSitContratoIV': 'NomSitContratoCUST','NomSituacaoContratoIII': 'NomSituacaoContratoCUSD','NomSitContratoII': 'NomSitContratoCCT'}
acesso_contratos = pd.read_parquet(os.path.join(directory,"vrapeelacesso.gzip"))[cols].rename(columns=rename_cols)
acesso_contratos = acesso_contratos.loc[acesso_contratos.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
del acesso_contratos['DthEnvio']
df_usina = pd.merge(df_usina,acesso_contratos,on="IdeUsinaOutorga",how="left")

##  Cria colunas com definições e regras

In [11]:
df_usina["classe"] = df_usina.CodCegFormatado.str.slice(4,6)
df_usina["CC_real"] = pd.to_datetime(pd.NA)
df_usina.loc[df_usina.classe == "PH","CC_real"] = df_usina.DatRealizacaoIV
df_usina.loc[df_usina.classe == "CV","CC_real"] = df_usina.DatRealizacaoV
df_usina["ME_real"] = df_usina.DatRealizacaoX
df_usina.loc[df_usina.classe == "CV","ME_real"] = df_usina.DatRealizacaoVI

In [12]:
dict_validades = {
    'DatValidadeLO' : 'LO',
    'DatValidadeLI' : 'LI',
    'DatValidadeLP' : "LP"
}

df_usina['ValidadeAmbiental'] = pd.NA
for validade in dict_validades:
    df_usina.loc[df_usina.ValidadeAmbiental.isna(),'ValidadeAmbiental'] = df_usina[validade]
df_usina[['IdeUsinaOutorga'] +list(dict_validades.keys()) + ['ValidadeAmbiental']].sample(5)

,IdeUsinaOutorga,DatValidadeLO,DatValidadeLI,DatValidadeLP,ValidadeAmbiental
1001,46719,NaT,2026-01-21,2025-01-15,2026-01-21
931,45724,2030-12-03,2030-12-03,2030-12-03,2030-12-03
1425,49187,2030-10-21,2030-10-21,2030-10-21,2030-10-21
1330,48593,2031-07-12,2031-07-12,2031-07-12,2031-07-12
2027,51787,NaT,2027-01-03,2025-12-11,2027-01-03


In [13]:
#WORK.dadosAcesso t1.'DthEnvio' LABEL="datadainformacao" AS datadainformacao

df_usina['CondicaoAmbiental'] = pd.NA

for validade in reversed(dict_validades):
    df_usina.loc[df_usina[validade].notna(),'CondicaoAmbiental'] = dict_validades[validade]

df_usina.loc[(df_usina.CondicaoAmbiental.isna()) & (df_usina.DthEnvio.notna()),'CondicaoAmbiental'] = 'Sem LP'
df_usina.loc[(df_usina.CondicaoAmbiental.isna()),'CondicaoAmbiental'] = 'Não Informado'


df_usina[list(dict_validades) + ['ValidadeAmbiental','CondicaoAmbiental']].sample(10)

display(df_usina[list(dict_validades.keys()) + ['CondicaoAmbiental','ValidadeAmbiental']])

df_usina['CondicaoConexao'] = "Não informado"

nsa = "Não se Aplica"
nass = "Não Assinado"
valido = "Válido"
vencido = "Vencido"

df_usina.loc[
    ((df_usina.NomSitContratoCCD ==  nsa)&
    (df_usina.NomSitContratoCCT == nsa) &
    df_usina.DthEnvio.notna()
    )
    ,'CondicaoConexao'] = "Verificar"

df_usina.loc[
(    ((df_usina.NomSitContratoCCD ==  valido) |
    (df_usina.NomSitContratoCCT == valido)) &
    df_usina.DthEnvio.notna())
    
    ,'CondicaoConexao'] = "OK"

df_usina.loc[
    (((df_usina.NomSitContratoCCD ==  vencido) |
    (df_usina.NomSitContratoCCT == vencido) |
    (df_usina.NomSitContratoCCD == nass) |
    (df_usina.NomSitContratoCCT == nass )) &
    df_usina.DthEnvio.notna())
    ,'CondicaoConexao'] = "Sem Conexão"
display(df_usina[['IdeUsinaOutorga'] + ['NomSitContratoCCD','NomSitContratoCCT','DthEnvio','CondicaoConexao']].sample(10))


df_usina['CondicaoUso'] = "Não informado"

df_usina.loc[
    ((df_usina.NomSituacaoContratoCUSD ==  nsa)&
    (df_usina.NomSitContratoCUST == nsa) 
    )
    ,'CondicaoUso'] = "Verificar"


df_usina.loc[
    ((df_usina.NomSituacaoContratoCUSD ==  valido) |
    (df_usina.NomSitContratoCUST == valido) 
    )
    ,'CondicaoUso'] = "OK"

df_usina.loc[
    (((df_usina.NomSituacaoContratoCUSD ==  vencido) |
    (df_usina.NomSitContratoCUST == vencido) |
    (df_usina.NomSituacaoContratoCUSD == nass) |
    (df_usina.NomSitContratoCUST == nass )))
    ,'CondicaoUso'] = "Sem Uso"

display(df_usina[['IdeUsinaOutorga'] + ['NomSituacaoContratoCUSD','NomSitContratoCUST','CondicaoConexao']].sample(10))


df_usina["PPA"] = "Ambos"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "ACR") & 
        df_usina.DatConclusaoACL.isna())),
    "PPA"] = "ACR"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "Fora do ACR") & 
        df_usina.DatConclusaoACL.isna())),
    "PPA"] = "Nenhum"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "Fora do ACR") & 
        df_usina.DatConclusaoACL.notna())),
    "PPA"] = "ACL"

display(df_usina[['IdeUsinaOutorga'] + ['DscComercializacaoEnergia','DatConclusaoACL','PPA']].sample(10))

,DatValidadeLO,DatValidadeLI,DatValidadeLP,CondicaoAmbiental,ValidadeAmbiental
0,NaT,2026-03-09,2002-12-11,LI,2026-03-09
1,NaT,NaT,NaT,Sem LP,NaT
2,2026-02-22,2028-06-28,2000-03-29,LO,2026-02-22
3,NaT,NaT,NaT,Não Informado,NaT
4,NaT,2009-03-10,1999-01-09,LI,2009-03-10
...,...,...,...,...,...
2472,NaT,NaT,NaT,Não Informado,NaT
2473,NaT,NaT,NaT,Não Informado,NaT
2474,NaT,NaT,NaT,Não Informado,NaT
2475,NaT,NaT,NaT,Não Informado,NaT


,IdeUsinaOutorga,NomSitContratoCCD,NomSitContratoCCT,DthEnvio,CondicaoConexao
1982,51608,Não se Aplica,Não Assinado,2023-01-05 15:00:11.713,Sem Conexão
2473,70559,NaN,NaN,NaT,Não informado
505,37861,Não se Aplica,Não Assinado,2023-01-10 10:24:10.320,Sem Conexão
191,33864,Não se Aplica,Válido,2023-01-04 11:46:40.117,OK
2132,52383,Não se Aplica,Não Assinado,2023-01-05 15:15:54.613,Sem Conexão
931,45724,Válido,Não se Aplica,2023-01-05 16:28:49.907,OK
829,44548,Não se Aplica,Válido,2023-01-10 11:20:40.370,OK
1732,49926,Não se Aplica,Não Assinado,2023-01-05 12:25:37.700,Sem Conexão
1493,49392,Não se Aplica,Válido,2023-01-05 19:16:37.960,OK
1839,50527,Não se Aplica,Não Assinado,2023-01-04 12:09:57.060,Sem Conexão


,IdeUsinaOutorga,NomSituacaoContratoCUSD,NomSitContratoCUST,CondicaoConexao
2124,52364,Não se Aplica,Não Assinado,Sem Conexão
1395,48918,Não se Aplica,Não Assinado,Sem Conexão
1928,51016,Não se Aplica,Não Assinado,Sem Conexão
2332,54845,Não se Aplica,Não se Aplica,Verificar
1961,51456,Válido,Não se Aplica,OK
1513,49418,Não se Aplica,Válido,OK
944,46508,Não se Aplica,Não Assinado,Sem Conexão
525,37928,NaN,NaN,Não informado
338,35676,Não se Aplica,Válido,OK
744,43273,Válido,Não se Aplica,OK


,IdeUsinaOutorga,DscComercializacaoEnergia,DatConclusaoACL,PPA
1726,49919,Fora do ACR,NaT,Nenhum
1574,49498,Fora do ACR,NaT,Nenhum
99,32145,Fora do ACR,NaT,Nenhum
2474,70560,Fora do ACR,NaT,Nenhum
1889,50866,Fora do ACR,2022-05-24,ACL
350,36925,ACR,NaT,ACR
817,44512,Fora do ACR,NaT,Nenhum
1836,50524,Fora do ACR,NaT,Nenhum
838,44560,Fora do ACR,NaT,Nenhum
1430,49192,Fora do ACR,NaT,Nenhum


In [14]:
ea = "Em andamento"
na = "Não Iniciada"
nenhum = "Nenhum"
paralisada = "Paralisada"
ok = "OK"

In [15]:
slp = "Sem LP"
sim = "Sim"
df_usina['criterio_novo'] = np.select(
    [
        (df_usina.IdcSemPrevisao == sim),
        (df_usina.IdcSituacaoObra == ea),
        (( df_usina.IdcSituacaoObra == na)  &  (df_usina.PPA != nenhum) ),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso != ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso != ok)),
        (df_usina.IdcSituacaoObra == paralisada),
        (df_usina.CondicaoAmbiental == "Sem LP"),
        ((df_usina.DthEnvio.isna()) & (df_usina.PPA == nenhum)),
        
    ],
    [9,0,1,2,4,3,5,8,6,7],
    default=-8
)


display(df_usina[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio_novo']].sample(5))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio_novo
1249,47440,Não Iniciada,Nenhum,LP,OK,4
2359,55077,Não Iniciada,Nenhum,LP,Sem Uso,5
1056,46924,Não Iniciada,Nenhum,LP,OK,4
2126,52366,Não Iniciada,Nenhum,LP,Sem Uso,5
1584,49508,Não Iniciada,Nenhum,LO,OK,2


In [16]:
dict_marcos_homologar = {
    'DatInicioObraRealizado' : 'IO_real',
    'DatConcretagemRealizado' : 'CC_real',
    'DatMontagemRealizado' : 'ME_real',
    'DatEnchimentoRealizado' : 'Ench_Real',
    'DatSisTransmissaoRealizado' : 'IOTrans_Real',
    'DatCanteiroObraRealizado' : 'canteiroReal',
    'DatComissionamentoUGRealizado' : 'Comiss_Real',
    'DatConclusaoSisTransRealizado' : 'Conc_Trans_REAL',
    'DatDesvioRioRealizado' : 'DesvRio_real',
    'DatConclusaoTorresRealizado' : 'ME_Real_conc_eol'}
    
df_usina['Homologar_Marcos'] = False
df_usina['Dsc_Marcos_a_Homologar'] = ''


for marco_monitoramento, marco_rapeel in dict_marcos_homologar.items():
    mask = (df_usina[marco_monitoramento].isna()) & (df_usina[marco_rapeel].notna())
    df_usina['Homologar_Marcos'] |= mask
    df_usina.loc[mask,'Dsc_Marcos_a_Homologar'] += f'{marco_monitoramento}, '

df_usina.Dsc_Marcos_a_Homologar = df_usina.Dsc_Marcos_a_Homologar.str.slice(0,-2)

In [17]:
"""df_usina['Homologar_Marcos'] =  np.select(
    [((df_usina['DatInicioObraRealizado'].isna()) & (df_usina['IO_real'].notna())) |
     ((df_usina['DatConcretagemRealizado'].isna()) & (df_usina['CC_real'].notna())) | 
     ((df_usina['DatMontagemRealizado'].isna()) & (df_usina['ME_real'].notna())) | 
     ((df_usina['DatEnchimentoRealizado'].isna()) & (df_usina['Ench_Real'].notna())) | 
     ((df_usina['DatSisTransmissaoRealizado'].isna()) & (df_usina['IOTrans_Real'].notna())) |
      ((df_usina['DatCanteiroObraRealizado'].isna()) & (df_usina['canteiroReal'].notna())) |
     ((df_usina['DatComissionamentoUGRealizado'].isna()) & (df_usina['Comiss_Real'].notna())) | 
     ((df_usina['DatConclusaoSisTransRealizado'].isna()) & (df_usina['Conc_Trans_REAL'].notna())) | 
     ((df_usina['DatDesvioRioRealizado'].isna()) & (df_usina['DesvRio_real'].notna())) |
     ((df_usina['DatConclusaoTorresRealizado'].isna()) & (df_usina['ME_Real_conc_eol'].notna()))],

    [True],
    default= False
)
"""


df_usina['Revisar_IO'] =  np.select(
 [(df_usina.DatPrevisaoIniciobra < (hoje + pd.Timedelta(15,"D"))) |
 (df_usina.prev_IO > df_usina.DatPrevisaoIniciobra)],
 [True],
 default = False)

In [18]:
usinas_selecionadas = read_file("./usinas_selecionadas.txt").split("\n")
usinas_selecionadas = [int(num) for num in usinas_selecionadas]
df_usina["Usina_Selecionada"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(usinas_selecionadas),'Usina_Selecionada'] = True

df_usina['Paralisada'] = False
df_usina.loc[df_usina.IdcObraParalisada=="Sim",'Paralisada'] = True

df_usina['Sem_Previsao'] = False
df_usina.loc[df_usina.IdcSemPrevisao=="Sim",'Sem_Previsao'] = True

# Informações de UGs

## Carrega banco de dados

In [19]:
def ultimo_rapeel(df):
    df_ultimo_envio = df.loc[df.groupby("IdeUsinaOutorga")['DthEnvio'].idxmax()][['IdeUsinaOutorga','DthEnvio']]
    return pd.merge(df_ultimo_envio,df,on=["IdeUsinaOutorga","DthEnvio"],how="left")

In [20]:
cols = ["IdcMonitorada","IdeUsinaOutorga",'NumUgUsina','DatLiberOpComerRealizado']
monitoramentoug = pd.read_parquet(os.path.join(directory,"vmonitoramentoug.gzip"))[cols].rename(columns={'NumUgUsina':'NumOperacaoUg'})
monitoramentoug.IdcMonitorada = monitoramentoug.IdcMonitorada.map({"Sim":True,"Não":False}).astype(bool)

In [21]:
# Carrega BD vrapeeloperacaoug => ug_rapeel
cols = ["IdeUsinaOutorga",'NumOperacaoUg',"DthEnvio",'DatPrevistaComercial']
ug_rapeel = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols]
ug_rapeel = ultimo_rapeel(ug_rapeel)

In [22]:
list_cols_used =   ['IdeUsinaOutorga','IdcObraParalisada','IdcSemPrevisao','criterio_novo',] # 'DscJustificativaPrevisao'
df_ug = pd.merge(df_usina[list_cols_used],monitoramentoug[monitoramentoug.IdcMonitorada],on="IdeUsinaOutorga",how='left')
df_ug.IdcMonitorada = df_ug.IdcMonitorada.astype(bool)
df_ug = pd.merge(df_ug,ug_rapeel,on=list_id_ug,how='left') # poderia ser inner?
df_ug = df_ug.loc[df_ug.DatLiberOpComerRealizado.isna() & df_ug.NumOperacaoUg.notna()].reset_index(drop=True)

In [23]:
previsao_path = os.path.join(get_standard_folder_path("Documents"),r"Previsor\Previsoes")
previsao_file = sorted(glob.glob(f"{previsao_path}/*.gzip"))[-1]
calculo_previsao = pd.read_parquet(previsao_file)
calculo_previsao.FaseAtual =calculo_previsao.FaseAtual.str.slice(0,-3)
calculo_previsao.Indicador =calculo_previsao.Indicador/100

In [24]:
cols = ["Dat_OC_obrigacao","DatPrevisaoIniciobra","FASE","Ind_crono_norm","IdeUsinaOutorga","flagOPTeste30dias",'Previsao_OC']
rename_cols = {'FaseAtual':'FASE','Indicador':'Ind_crono_norm','NumUgUsina':'NumOperacaoUg'}
calculo_previsao = calculo_previsao.rename(columns=rename_cols)[['IdeUsinaOutorga','NumOperacaoUg','FASE','Ind_crono_norm','flagOPTeste30dias','Previsao_OC','Dat_OC_obrigacao','MdaPotenciaUnitaria']] 

In [25]:
df_ug = pd.merge(df_ug,calculo_previsao,on=list_id_ug,how="left")

In [26]:
monitoramentoug['Existe_Monitoramento'] = True     # Adiciona colunas para checagem
ug_rapeel['Existe_Rapeel'] = True                  # dos resultados

comp_ug = pd.merge(monitoramentoug[list_id_ug + ['Existe_Monitoramento',"IdcMonitorada"]], # Faz merge tipo outer (todas as UGs) com os dados do  
        ug_rapeel[list_id_ug + ['Existe_Rapeel']], how="outer")                            # monitoramento e rapeel

# Inner join com usinas monitoradas
comp_ug = pd.merge(df_usina[df_usina.IdcUsinaMonitorada][['IdeUsinaOutorga']],comp_ug,how="inner",on="IdeUsinaOutorga")

# Inner join com usinas que já enviaram rapeel
comp_ug = pd.merge(rapeel_cronograma[['IdeUsinaOutorga']],comp_ug,how="inner",on="IdeUsinaOutorga") 

# Das UGs que estão no monitoramento, remove-se UGs não monitoradas
comp_ug = comp_ug[comp_ug.IdcMonitorada != False]

# Preenche com o valor falso as usinas que não foram encontradas
comp_ug[['Existe_Monitoramento','Existe_Rapeel']] = comp_ug[['Existe_Monitoramento','Existe_Rapeel']].fillna(False) 

# Coluna com diferenças
comp_ug["Diff"] = ~(comp_ug.Existe_Monitoramento & comp_ug.Existe_Rapeel) 

# Remove casos que não tem diferença
comp_ug = comp_ug[comp_ug.Diff]#.drop_duplicates(subset=['IdeUsinaOutorga','Existe_Monitoramento','IdcMonitorada','Existe_Rapeel','Diff'])

df_UG_Sem_Rapeel = comp_ug[comp_ug.Existe_Monitoramento][['IdeUsinaOutorga']].drop_duplicates()
df_UG_Sem_Rapeel['UG_Sem_Rapeel'] = True


df_ug = pd.merge(df_ug,df_UG_Sem_Rapeel,on='IdeUsinaOutorga',how='left')


df_ug.loc[df_ug.UG_Sem_Rapeel.isna(),'UG_Sem_Rapeel'] = False

df_ug = df_ug.merge(df_ug.loc[df_ug[df_ug.DatPrevistaComercial.notna()].groupby("IdeUsinaOutorga").DatPrevistaComercial.idxmax()][['IdeUsinaOutorga','DatPrevistaComercial']].rename(columns={'DatPrevistaComercial':'PrevisaoOC_rapeel_max'}),
            how="left",on="IdeUsinaOutorga")

df_ug.loc[df_ug.UG_Sem_Rapeel,'DatPrevistaComercial'] = df_ug.PrevisaoOC_rapeel_max

##  Cria colunas com definições e regras

In [27]:
################################ criterionovopmo ################################
list_condicoes = [
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] != "OT")),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1)),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT")),
    ((df_ug['criterio_novo'] == 1)),
    ((df_ug['criterio_novo'] == 2)),
    ((df_ug['criterio_novo'] == 3)),
    ((df_ug['criterio_novo'] == 5)),
    ((df_ug['criterio_novo'] == 4)),
    ((df_ug['criterio_novo'] == 6)),
    ((df_ug['criterio_novo'] == 7)),
    ((df_ug['criterio_novo'] == 8)),
    ((df_ug['criterio_novo'] == 9)),
]

list_values = [0.1, 0.2, 0.3, 1, 2, 3, 5, 4, 6, 7, 8, 9]

df_ug['criterionovopmo'] = np.select(list_condicoes,list_values)

################################ regranovapmo ################################

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] != "OT"),'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC']].max(axis=1)


df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1),'regranovapmo'] = ( hoje + pd.Timedelta(60,"D"))

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['Previsao_OC'].isna()),'regranovapmo'] = df_ug['DatPrevistaComercial']


df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['Previsao_OC'].notna()),'regranovapmo'] = df_ug['Previsao_OC']

df_ug.loc[(df_ug['criterio_novo']== 1) | (df_ug['criterio_novo']== 2) ,'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)
df_ug["Dummy"] = hoje + pd.Timedelta(5*365,'D')

df_ug.loc[((df_ug['criterio_novo'] == 3) | (df_ug['criterio_novo'] == 4) | (df_ug['criterio_novo'] == 5) ),'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)


df_ug.loc[df_ug['criterio_novo']== 6,'regranovapmo'] = pd.NA

df_ug["Dummy"] = hoje + pd.Timedelta(6*365,'D')
df_ug.loc[df_ug['criterio_novo']== 7,'regranovapmo'] =  df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)


df_ug.loc[df_ug['criterio_novo']== 8,'regranovapmo'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)

df_ug.loc[(df_ug['criterio_novo'] == 9),'regranovapmo'] = pd.NA


In [28]:
serie_usinas_previsao_OC_passado = df_ug[(df_ug.Previsao_OC < final_do_mes) | (df_ug.Previsao_OC.isna())].IdeUsinaOutorga.drop_duplicates()
serie_usinas_previsao_em_teste = df_ug[df_ug.criterionovopmo.isin([0.2,0.3])].IdeUsinaOutorga.drop_duplicates()

serie_flagOPTeste30dias = df_ug[df_ug.flagOPTeste30dias == 1].IdeUsinaOutorga.drop_duplicates()
df_usina["flagOPTeste30dias"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_flagOPTeste30dias),'flagOPTeste30dias'] = True

serie_UG_Sem_Rapeel = df_ug[df_ug.UG_Sem_Rapeel].IdeUsinaOutorga.drop_duplicates()

In [29]:
df_usina["UG_Sem_Rapeel"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_UG_Sem_Rapeel),'UG_Sem_Rapeel'] = True

df_usina["flagOPTeste30dias"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_flagOPTeste30dias),'flagOPTeste30dias'] = True

df_usina["Prev_OC_passado"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_usinas_previsao_OC_passado),'Prev_OC_passado'] = True

df_usina["Em_teste"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_usinas_previsao_em_teste),'Em_teste'] = True

df_usina["Sem_Monitoramento"] = False
df_usina.loc[df_usina.DatMonitoramento < (hoje - pd.Timedelta(4*30, unit="D")),'Sem_Monitoramento'] = True

In [30]:
df_usina["Manual"] = False
df_usina.loc[(df_usina.Paralisada | df_usina.Sem_Previsao | 
    df_usina.Usina_Selecionada| df_usina.Prev_OC_passado | df_usina.Em_teste | 
    (df_usina.DscJustificativaPrevisao == 'Situação das obras de conexão e linha de transmissão associada.')),
    'Manual'] = True

In [31]:
df_usina['Caso_I'] = df_usina['Caso_II_a'] = df_usina['Caso_II_b'] = df_usina['Caso_III'] = False
list_casos = ['Caso_I','Caso_II_a','Caso_II_b','Caso_III']

In [32]:
df_usina['Caso_I'] = (~df_usina.Manual) & (~df_usina.Revisar_IO) & (~ df_usina.Homologar_Marcos) 

df_usina['Caso_II_a'] = (~df_usina.Manual) & (df_usina.Revisar_IO) & (~ df_usina.Homologar_Marcos)

df_usina['Caso_II_b'] = (~df_usina.Manual)  & ((df_usina.Homologar_Marcos) | (df_usina.Homologar_Marcos & df_usina.Revisar_IO))

df_usina['Caso_III'] = (df_usina.Manual & (df_usina.Sem_Monitoramento | df_usina.Homologar_Marcos | df_usina.Revisar_IO)) | df_usina.Em_teste


df_usina['Selecionado_BIU'] = df_usina[list_casos].any(axis=1)

In [33]:
list_criterios = ['criterio_novo',"criterionovopmo"]

df_usina_criterio = df_ug.loc[df_ug.groupby("IdeUsinaOutorga").criterionovopmo.idxmax()][["IdeUsinaOutorga","FASE",'flagOPTeste30dias'] + list_criterios]

################################ dscjustificativaregranova ################################
list_condicoes = [
    (df_usina_criterio['criterionovopmo'] == 9),
    (df_usina_criterio['criterionovopmo'] == 8),
    (df_usina_criterio['criterionovopmo'] == 0.2),
    (df_usina_criterio['criterionovopmo'] == 0.1) | (df_usina_criterio['criterionovopmo'] == 0.3),
    (df_usina_criterio['criterionovopmo'] == 1),
    df_usina_criterio['criterionovopmo']==2,
    df_usina_criterio['criterionovopmo']==3,
    (df_usina_criterio.criterionovopmo== 4),
    (df_usina_criterio.criterionovopmo== 5),
    (df_usina_criterio.criterionovopmo== 6),
    (df_usina_criterio.criterionovopmo== 7),
]

list_values = [
    "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina",
    "Paralisação de obras",
    "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras",
    "Estágio das obras da usina",
    "Compromisso de venda de energia - PPA",
    "Acesso a rede contratado - CUST/CUSD",
    "Acesso a rede não contratado - CUST/CUSD",
    "Sem Licença de Instalação - LI",
    "Sem Licença de Instalação - LI",
    "Nenhuma licença ambiental válida",
    "Sem RAPEEL"
]

df_usina_criterio['dscjustificativaregranova'] = np.select(list_condicoes,list_values)

################################ dsccriterionovo ################################
list_condicoes = [
    ((df_usina_criterio['criterio_novo'] == 9)),
    ((df_usina_criterio['criterio_novo'] == 0) & (df_usina_criterio['FASE'] != "OT")),
    ((df_usina_criterio['criterio_novo'] == 0) & (df_usina_criterio['FASE'] == "OT") & (df_usina_criterio['flagOPTeste30dias'] == 1)),
    ((df_usina_criterio['criterio_novo'] == 0) & (df_usina_criterio['FASE'] == "OT")),
    ((df_usina_criterio['criterio_novo'] == 1)),
    ((df_usina_criterio['criterio_novo'] == 2)),
    ((df_usina_criterio['criterio_novo'] == 3)),
    ((df_usina_criterio['criterio_novo'] == 4)),
    ((df_usina_criterio['criterio_novo'] == 5)),
    ((df_usina_criterio['criterio_novo'] == 7)),
    ((df_usina_criterio['criterio_novo'] == 6)),
    ((df_usina_criterio['criterio_novo'] == 8))
]

list_values = [
    "Usina Viabilidade Baixa análise da fiscalização",
    "Usina em obras = Previsão OC maior entre data calculada e data RAPEEL",
    "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias",
    "Usina em Teste = Previsão OC conforme data calculada",
    "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras, sem cust, sem PPA e com LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, com cust, sem PPA e sem LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem cust, sem PPA e sem LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem PPA e sem RAPEEL  = Previsão OC handicap de 5 anos",
    "Usina sem LP = Previsão OC sem previsão","Usina obras paralisadas = Previsão OC maior entre data RAPEEL e data compromisso"
]
df_usina_criterio['dsccriterionovo'] = np.select(list_condicoes,list_values)
list_justificativas = ['dsccriterionovo','dscjustificativaregranova']

In [34]:
df_usina = pd.merge(df_usina,
df_usina_criterio[['IdeUsinaOutorga','criterionovopmo'] + list_justificativas],
on = "IdeUsinaOutorga", how='left')

In [35]:
list_casos = ['Caso_I','Caso_II_a','Caso_II_b','Caso_III','Selecionado_BIU']
df_ug = pd.merge(df_usina[['IdeUsinaOutorga','dscjustificativaregranova','dsccriterionovo','DatMonitoramento','DscJustificativaPrevisao'] + list_casos],df_ug, on = "IdeUsinaOutorga",how='left')


## Análise de casos BIU

In [36]:
list_criterios_BIU = ['Manual', 'Revisar_IO','Homologar_Marcos', 'Em_teste','Sem_Monitoramento']
casos = df_usina[list_criterios_BIU +  list_casos].value_counts().reset_index().sort_values(by='Caso_III')
casos
mask_casos_selecionados = casos[list_casos].any(axis=1)
casos[mask_casos_selecionados]

,Manual,Revisar_IO,Homologar_Marcos,Em_teste,Sem_Monitoramento,Caso_I,Caso_II_a,Caso_II_b,Caso_III,Selecionado_BIU,0
0,False,False,False,False,False,True,False,False,False,True,2062
2,False,False,True,False,False,False,False,True,False,True,80
3,False,True,False,False,False,False,True,False,False,True,79
6,False,True,True,False,False,False,False,True,False,True,19
9,False,False,False,False,True,True,False,False,False,True,1
4,True,False,False,True,False,False,False,False,True,True,56
5,True,False,False,False,True,False,False,False,True,True,41
7,True,False,True,False,False,False,False,False,True,True,8
8,True,False,True,True,False,False,False,False,True,True,3


In [37]:
list_carac_usina = ['Usina_Sem_Rapeel','CondicaoAmbiental','Homologar_Marcos','Revisar_IO','Usina_Selecionada','Paralisada','Sem_Previsao','criterionovopmo','Prev_OC_passado','Em_teste','Sem_Monitoramento','Manual','UG_Sem_Rapeel']
list_carac_ug = ['FASE','flagOPTeste30dias','UG_Sem_Rapeel','criterionovopmo']
list_casos = ['Caso_I','Caso_II_a','Caso_II_b','Caso_III']
list_ambiental = ['DatValidadeLI','DatValidadeLP','DatValidadeLO','ValidadeAmbiental','CondicaoAmbiental']
list_contratos = ['NomSitContratoCCD', 'NomSitContratoCUST', 'NomSituacaoContratoCUSD', 'NomSitContratoCCT']

In [38]:
df_usina.loc[df_usina.Selecionado_BIU,list_casos+ ['Manual','Homologar_Marcos']].value_counts().reset_index()

,Caso_I,Caso_II_a,Caso_II_b,Caso_III,Manual,Homologar_Marcos,0
0,True,False,False,False,False,False,2063
1,False,False,True,False,False,True,99
2,False,False,False,True,True,False,97
3,False,True,False,False,False,False,79
4,False,False,False,True,True,True,11


In [39]:
dic = {
    'NumOperacaoUg':'NumUgUsina',
    'regranovapmo': 'PrevisaoOC_regra',
    'dscjustificativaregranova' : 'Justificativadaprevisao_new',
    'criterionovopmo' : 'CriterioPrevisao',
    'dsccriterionovo':'DscCriterioPrevisao',
    'Previsao_OC':'CalculoPrevisorOC',
    'Dat_OC_obrigacao':'OC_Obrigacao',
    'DatPrevistaComercial':'PrevisaoOC_rapeel_max', 
    'DscJustificativaPrevisao':'DscJustificativaPrevisaoAtual'
}   

df_ug.rename(columns=dic)[['IdeUsinaOutorga','NumUgUsina','MdaPotenciaUnitaria','PrevisaoOC_regra','Justificativadaprevisao_new','CriterioPrevisao','DscCriterioPrevisao','CalculoPrevisorOC','OC_Obrigacao','PrevisaoOC_rapeel_max','DscJustificativaPrevisaoAtual','DatMonitoramento','DthEnvio','FASE',]]

,IdeUsinaOutorga,NumUgUsina,MdaPotenciaUnitaria,PrevisaoOC_regra,Justificativadaprevisao_new,CriterioPrevisao,DscCriterioPrevisao,CalculoPrevisorOC,OC_Obrigacao,PrevisaoOC_rapeel_max,PrevisaoOC_rapeel_max,DscJustificativaPrevisaoAtual,DatMonitoramento,DthEnvio,FASE
0,324,1.0,7500.0,NaT,Analisar justificativa: Revogação da outorga e...,9.0,Usina Viabilidade Baixa análise da fiscalização,2023-10-18 22:17:01.762,2015-06-30,2026-07-01,2026-07-01,Inviabilidade da implantação da usina,2022-10-13 11:01:56.560,2023-01-02 08:34:07.347,OUT
1,601,3.0,5000.0,NaT,Analisar justificativa: Revogação da outorga e...,9.0,Usina Viabilidade Baixa análise da fiscalização,2025-08-11 12:01:59.696,2003-10-31,2027-05-01,2027-05-01,Nenhuma licença ambiental válida,2022-10-13 11:03:48.340,2023-01-03 19:22:43.140,OUT
2,601,1.0,5000.0,NaT,Analisar justificativa: Revogação da outorga e...,9.0,Usina Viabilidade Baixa análise da fiscalização,2025-08-11 12:01:59.696,2003-10-31,2027-05-01,2027-05-01,Nenhuma licença ambiental válida,2022-10-13 11:03:48.340,2023-01-03 19:22:43.140,OUT
3,601,2.0,5000.0,NaT,Analisar justificativa: Revogação da outorga e...,9.0,Usina Viabilidade Baixa análise da fiscalização,2025-08-11 12:01:59.696,2003-10-31,2027-05-01,2027-05-01,Nenhuma licença ambiental válida,2022-10-13 11:03:48.340,2023-01-03 19:22:43.140,OUT
4,659,4.0,648.0,NaT,Analisar justificativa: Revogação da outorga e...,9.0,Usina Viabilidade Baixa análise da fiscalização,2023-04-22 22:31:32.351,2021-01-01,2023-06-12,2023-06-12,Paralisação de obras,2022-12-13 17:04:26.787,2023-01-06 09:37:52.703,ME
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74660,70562,1.0,3333.0,2029-02-23,Sem RAPEEL,7.0,"Usina sem obras, sem PPA e sem RAPEEL = Previ...",2027-10-31 17:33:25.374,2027-04-08,NaT,NaT,Sem RAPEEL,2023-01-15 02:18:33.310,NaT,OUT
74661,70562,3.0,6666.0,2029-02-23,Sem RAPEEL,7.0,"Usina sem obras, sem PPA e sem RAPEEL = Previ...",2027-10-31 17:33:25.374,2027-04-08,NaT,NaT,Sem RAPEEL,2023-01-15 02:18:33.310,NaT,OUT
74662,70562,6.0,6666.0,2029-02-23,Sem RAPEEL,7.0,"Usina sem obras, sem PPA e sem RAPEEL = Previ...",2027-10-31 17:33:25.374,2027-04-08,NaT,NaT,Sem RAPEEL,2023-01-15 02:18:33.310,NaT,OUT
74663,70562,4.0,6666.0,2029-02-23,Sem RAPEEL,7.0,"Usina sem obras, sem PPA e sem RAPEEL = Previ...",2027-10-31 17:33:25.374,2027-04-08,NaT,NaT,Sem RAPEEL,2023-01-15 02:18:33.310,NaT,OUT


In [40]:
dic = {'DatPrevisaoIniciobra':'prev_IO_SFG',
        'prev_IO':'prev_IO_rapeel'}
df_usina.rename(columns=dic)[['IdeUsinaOutorga','DatInicioObraOutorgado','prev_IO_rapeel','prev_IO_SFG','DatMonitoramento','DthEnvio']]

,IdeUsinaOutorga,DatInicioObraOutorgado,prev_IO_rapeel,prev_IO_SFG,DatMonitoramento,DthEnvio
0,324,2014-02-28,2025-02-01,NaT,2022-10-13 11:01:56.560,2023-01-02 08:34:07.347
1,601,2001-01-30,2026-05-29,NaT,2022-10-13 11:03:48.340,2023-01-03 19:22:43.140
2,659,2002-04-30,2023-02-10,NaT,2022-12-13 17:04:26.787,2023-01-06 09:37:52.703
3,1195,2002-03-31,NaT,NaT,2022-10-13 11:06:18.107,NaT
4,1358,2014-08-30,2023-02-01,NaT,2022-11-10 16:22:08.193,2022-07-05 18:10:51.733
...,...,...,...,...,...,...
2472,70558,NaT,NaT,2026-11-08,2023-01-15 02:15:48.717,NaT
2473,70559,NaT,NaT,2026-11-08,2023-01-15 02:16:29.043,NaT
2474,70560,NaT,NaT,2025-11-08,2023-01-15 02:17:11.490,NaT
2475,70561,NaT,NaT,2026-11-08,2023-01-15 02:17:52.910,NaT


In [41]:
#df_usina[df_usina.Homologar_Marcos & (~ df_usina.Sem_Monitoramento)][['IdeUsinaOutorga','NomUsina','Homologar_Marcos',"Sem_Monitoramento",'DatMonitoramento','DthEnvio','Dsc_Marcos_a_Homologar']].to_excel("Usinas_sem_fotos.xlsx",index=False)

# Testes

## Junta dados do SAS e Python

In [42]:
criterios_da_previsao_sas = load_pickle('./criterios_da_previsao_sas.pickle')
tabela_criterio_sas = load_pickle('./tabela_criterio_sas.pickle')
criterios_da_previsao_sas = pd.merge(criterios_da_previsao_sas,tabela_criterio_sas,on="IdeUsinaOutorga").rename(columns={'NumUgUsina':'NumOperacaoUg'})[["IdeUsinaOutorga","NumOperacaoUg",'CriterioNovoPMO','DscJustificativaRegranova','DscCriterioNovo','CalculoPrevisorOC_mx',"FASE"]]

criterios_da_previsao_sas['SAS'] = True
criterios_da_previsao_sas.CalculoPrevisorOC_mx = criterios_da_previsao_sas.CalculoPrevisorOC_mx.dt.normalize()
df_ug["PY"] = True


#criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x0020_',' ')
#criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x002F_','/')

criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x002F_','/')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0028_','(')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0029_',')')

criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x002F_','/')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0028_','(')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0029_',')')

criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x003D_','=')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x002C_', ',')

In [43]:
criterios_da_previsao_sas_gb = criterios_da_previsao_sas.groupby("IdeUsinaOutorga")[['CriterioNovoPMO','DscCriterioNovo','DscJustificativaRegranova']].nunique()
usinas_dois_criterios = criterios_da_previsao_sas_gb[criterios_da_previsao_sas_gb.CriterioNovoPMO>1].index

In [44]:
criterios_da_previsao_sas_usina = criterios_da_previsao_sas[['IdeUsinaOutorga','CriterioNovoPMO','DscJustificativaRegranova','DscCriterioNovo']].drop_duplicates()

In [45]:
merge_usina  = pd.merge(df_usina_criterio[['IdeUsinaOutorga','criterionovopmo','dscjustificativaregranova','dsccriterionovo']],
criterios_da_previsao_sas_usina,on=['IdeUsinaOutorga'],suffixes=('_py','_sas'),how='outer')

In [46]:
criterios_da_previsao_sas_ug = criterios_da_previsao_sas[list_id_ug + ['CalculoPrevisorOC_mx','FASE']]

In [47]:
merge_ug = pd.merge(df_ug[list_id_ug + ['FASE','Previsao_OC']],criterios_da_previsao_sas_ug,on=['IdeUsinaOutorga','NumOperacaoUg'],suffixes=('_py','_sas'),how='outer')
merge_ug['OC_diff'] = merge_ug.Previsao_OC - merge_ug.CalculoPrevisorOC_mx

## Teste previsão

In [48]:
merge_ug.sort_values(by='OC_diff',ascending=False).head(20)

,IdeUsinaOutorga,NumOperacaoUg,FASE_py,Previsao_OC,CalculoPrevisorOC_mx,FASE_sas,OC_diff
118,30114,1.0,OUT,2024-08-16 17:05:21.054,2021-12-01,OUT,989 days 17:05:21.054000
137,30414,1.0,OUT,2025-03-18 10:06:43.885,2022-09-08,OUT,922 days 10:06:43.885000
117,30114,2.0,OUT,2024-08-29 23:24:51.890,2022-05-19,OUT,833 days 23:24:51.890000
143,30612,1.0,OUT,2025-03-29 02:30:59.846,2023-01-23,OUT,796 days 02:30:59.846000
119,30114,3.0,OUT,2024-09-05 14:34:37.308,2022-08-11,OUT,756 days 14:34:37.308000
11575,37731,2.0,OUT,2024-09-28 19:25:41.359,2023-02-24,OUT,582 days 19:25:41.359000
11576,37731,3.0,OUT,2024-09-28 19:25:41.359,2023-02-24,OUT,582 days 19:25:41.359000
11577,37731,1.0,OUT,2024-09-28 19:25:41.359,2023-02-24,OUT,582 days 19:25:41.359000
11571,37730,2.0,OUT,2024-10-04 03:37:49.340,2023-05-16,OUT,507 days 03:37:49.340000
11572,37730,3.0,OUT,2024-10-04 03:37:49.340,2023-05-16,OUT,507 days 03:37:49.340000


## Teste critérios

### FASE ok

In [49]:
merge_ug[merge_ug.FASE_py != merge_ug.FASE_sas]

,IdeUsinaOutorga,NumOperacaoUg,FASE_py,Previsao_OC,CalculoPrevisorOC_mx,FASE_sas,OC_diff
12,1928,NaN,NaN,NaT,NaT,NaN,NaT
221,31691,4.0,OT,2023-02-20 00:37:48.805,2022-06-09,IO,256 days 00:37:48.805000
222,31691,7.0,OT,2023-02-20 00:37:48.805,2022-06-09,IO,256 days 00:37:48.805000
224,31691,5.0,OT,2023-02-20 00:37:48.805,2022-06-09,IO,256 days 00:37:48.805000
225,31691,8.0,OT,2023-02-20 00:37:48.805,2022-06-09,IO,256 days 00:37:48.805000
...,...,...,...,...,...,...,...
74744,50464,5.0,NaN,NaT,2023-02-26,OT,NaT
74745,50464,6.0,NaN,NaT,2023-02-26,OT,NaT
74746,50464,7.0,NaN,NaT,2023-02-26,OT,NaT
74747,1928,2.0,NaN,NaT,2023-03-19,OT,NaT


### criterionovopmo OK

In [50]:
merge_usina[(merge_usina.CriterioNovoPMO != merge_usina.criterionovopmo) & (~merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))]

,IdeUsinaOutorga,criterionovopmo,dscjustificativaregranova,dsccriterionovo,CriterioNovoPMO,DscJustificativaRegranova,DscCriterioNovo
86,31691,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
129,32548,7.0,Sem RAPEEL,"Usina sem obras, sem PPA e sem RAPEEL = Previ...",NaN,NaN,NaN
130,32550,7.0,Sem RAPEEL,"Usina sem obras, sem PPA e sem RAPEEL = Previ...",NaN,NaN,NaN
185,33762,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
247,34499,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
382,37314,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
424,37550,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
503,37833,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
640,40626,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
688,40872,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...


In [51]:
merge_usina[(merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))].sample(6)

,IdeUsinaOutorga,criterionovopmo,dscjustificativaregranova,dsccriterionovo,CriterioNovoPMO,DscJustificativaRegranova,DscCriterioNovo
638,40613,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
174,33549,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
771,44363,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
372,37101,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...
569,38106,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...
557,38081,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...


### dscjustificativaregranova OK

In [52]:
merge_usina[(merge_usina.dscjustificativaregranova != merge_usina.DscJustificativaRegranova) & (~merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))]


,IdeUsinaOutorga,criterionovopmo,dscjustificativaregranova,dsccriterionovo,CriterioNovoPMO,DscJustificativaRegranova,DscCriterioNovo
129,32548,7.0,Sem RAPEEL,"Usina sem obras, sem PPA e sem RAPEEL = Previ...",NaN,NaN,NaN
130,32550,7.0,Sem RAPEEL,"Usina sem obras, sem PPA e sem RAPEEL = Previ...",NaN,NaN,NaN
424,37550,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
503,37833,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
640,40626,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
862,44586,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
1454,49231,7.0,Sem RAPEEL,"Usina sem obras, sem PPA e sem RAPEEL = Previ...",NaN,NaN,NaN
1455,49232,7.0,Sem RAPEEL,"Usina sem obras, sem PPA e sem RAPEEL = Previ...",NaN,NaN,NaN
1456,49233,7.0,Sem RAPEEL,"Usina sem obras, sem PPA e sem RAPEEL = Previ...",NaN,NaN,NaN
1457,49235,7.0,Sem RAPEEL,"Usina sem obras, sem PPA e sem RAPEEL = Previ...",NaN,NaN,NaN


In [53]:
merge_usina[(merge_usina.dscjustificativaregranova != merge_usina.DscJustificativaRegranova)]

,IdeUsinaOutorga,criterionovopmo,dscjustificativaregranova,dsccriterionovo,CriterioNovoPMO,DscJustificativaRegranova,DscCriterioNovo
98,32102,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
129,32548,7.0,Sem RAPEEL,"Usina sem obras, sem PPA e sem RAPEEL = Previ...",NaN,NaN,NaN
130,32550,7.0,Sem RAPEEL,"Usina sem obras, sem PPA e sem RAPEEL = Previ...",NaN,NaN,NaN
133,32568,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...
139,32580,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...
172,33548,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...
175,33549,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...
277,34937,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
278,34937,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
372,37101,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...


### dsccriterionovo OK

In [54]:
merge_usina[(merge_usina.dsccriterionovo != merge_usina.DscCriterioNovo) & (~merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))]

,IdeUsinaOutorga,criterionovopmo,dscjustificativaregranova,dsccriterionovo,CriterioNovoPMO,DscJustificativaRegranova,DscCriterioNovo
86,31691,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
129,32548,7.0,Sem RAPEEL,"Usina sem obras, sem PPA e sem RAPEEL = Previ...",NaN,NaN,NaN
130,32550,7.0,Sem RAPEEL,"Usina sem obras, sem PPA e sem RAPEEL = Previ...",NaN,NaN,NaN
185,33762,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
247,34499,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
382,37314,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
424,37550,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
503,37833,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
640,40626,0.2,Analisar justificativa: Sem Licença de Operaçã...,Usina em Teste há mais de 30 dias = Previsão O...,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...
688,40872,0.3,Estágio das obras da usina,Usina em Teste = Previsão OC conforme data cal...,0.1,Estágio das obras da usina,Usina em obras = Previsão OC maior entre data ...
